In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler


2022-12-16 11:14:05,153 [15597] WARNING  py.warnings:109: [JupyterRequire] /Users/javi/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



In [2]:
auth_manager = SpotifyClientCredentials('74c040f7ab8049648ec083fdadfc1ddb','a9fa947f1cde460bb37b94060380a6cb')
sp = spotipy.Spotify(auth_manager=auth_manager)

In [3]:
playlist_16_dec = 'https://open.spotify.com/playlist/37i9dQZF1DX4JAvHpjipBk'

In [4]:
def spoti_info(playlist):
    play = sp.playlist_tracks(playlist, fields=None, limit=100,\
                          offset=0, market=None, additional_types=('track', 'episode'))
    links_track = []
    for e in (play['items']):
        links_track.append(e['track']['external_urls']['spotify'])
    
    audio_features = []
    for e in links_track:
        audio_features.append(sp.audio_features(e))
    
    track_song = []
    for e in (play['items']):
        track_song.append(e['track']['name'])
    
    artist_name = []
    for e in (play['items']):
        artist_name.append(e['track']['album']['artists'][0]['name'])
    
    audio_features=[e[0] for e in audio_features]
    
    datos_spotipy = pd.DataFrame(audio_features)
    
    datos_spotipy['artist_name'] = artist_name
    
    datos_spotipy['track_name'] = track_song
    
    datos_spotipy.drop(['type','id','uri','track_href','analysis_url','time_signature','key','mode'],axis=1, inplace=True)
    
    datos_spotipy=datos_spotipy.iloc[:, [10,11,1,0,2,4,3,6,7,8,9]]
    
    datos_spotipy.rename(columns = {'duration_ms':'duration'}, inplace = True)
    
    datos_spotipy['duration'] = (datos_spotipy['duration']/1000).round(2)





    return datos_spotipy

In [5]:
new_songs = spoti_info(playlist_16_dec)

In [6]:
new_songs

,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration
0,Simon Franglen,Nothing Is Lost (You Give Me Strength),0.693,0.520,-7.704,0.00020,0.0331,0.1220,0.120,126.985,267.50
1,Juice WRLD,Face 2 Face,0.720,0.711,-4.532,0.09480,0.0308,0.1010,0.437,155.997,124.56
2,ROSALÍA,DESPECHÁ RMX,0.582,0.954,-5.744,0.23500,0.0727,0.0572,0.884,130.008,170.83
3,Only The Family,Mad Max,0.640,0.898,-8.796,0.02120,0.2550,0.0919,0.680,149.988,153.60
4,Little Simz,Gorilla,0.813,0.716,-5.748,0.26900,0.2560,0.1040,0.682,90.418,245.82
...,...,...,...,...,...,...,...,...,...,...,...
95,young friend,little pieces,0.576,0.564,-9.626,0.21700,0.1310,0.6740,0.677,79.896,157.50
96,chemical club,Bounds,0.713,0.531,-7.337,0.00118,0.0324,0.3280,0.692,150.043,207.07
97,whiterosemoxie,whts rght,0.509,0.444,-9.560,0.34500,0.0881,0.1040,0.237,113.780,258.00
98,Mokita,High,0.570,0.669,-6.364,0.14400,0.0558,0.2660,0.299,75.029,166.83


In [ ]:
datos_spotipy_sp.to_csv('../CSV_full/datos_proyecto_final.csv', index=False)

In [ ]:
new_songs = new_songs.copy()

In [7]:
new_songs_norm = new_songs.copy()

In [8]:
new_songs_norm['artist_name']=LabelEncoder().fit_transform(new_songs_norm['artist_name'])


In [10]:
scaler=StandardScaler()

In [11]:
new_songs_norm[['energy','danceability','loudness','acousticness','speechiness','liveness',\
                  'valence','tempo','duration',
                    'artist_name']] = scaler.fit_transform(new_songs_norm[['energy',\
                    'danceability','loudness','acousticness','speechiness','liveness',\
                  'valence','tempo','duration','artist_name']])

In [12]:
new_songs_norm.drop(['track_name'],axis=1,inplace=True)

In [13]:
new_songs_norm

,artist_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration
0,1.056604,0.394380,-0.784257,-0.253231,-0.943513,-0.806403,-0.350057,-1.574873,0.119805,1.798328
1,-0.606248,0.532061,0.521698,0.865842,-0.568450,-0.826180,-0.558507,-0.171132,1.045548,-1.244677
2,0.710176,-0.171643,2.183202,0.438252,-0.012595,-0.465880,-0.993275,1.808275,0.216266,-0.259650
3,0.433034,0.124117,1.800304,-0.638485,-0.860254,1.101729,-0.648836,0.904921,0.853807,-0.626454
4,-0.225178,1.006297,0.555886,0.436841,0.122206,1.110328,-0.528729,0.913778,-1.047009,1.336789
...,...,...,...,...,...,...,...,...,...,...
95,1.714816,-0.202238,-0.483409,-0.931307,-0.083960,0.035445,5.129203,0.891637,-1.382755,-0.543428
96,1.541602,0.496366,-0.709045,-0.123754,-0.939628,-0.812422,1.694739,0.958060,0.855562,0.511852
97,1.680173,-0.543892,-1.303904,-0.908023,0.423526,-0.333454,-0.528729,-1.056773,-0.301552,1.596085
98,0.155892,-0.232834,0.234525,0.219518,-0.373385,-0.611204,1.079315,-0.782225,-1.538056,-0.344804


In [17]:
new_songs_norm.to_csv('../Viernes//new_songs_norm.csv', index=False)

In [18]:
new_songs.to_csv('../Viernes/new_songs.csv', index=False)

In [15]:
new_songs

,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration
0,Simon Franglen,Nothing Is Lost (You Give Me Strength),0.693,0.520,-7.704,0.00020,0.0331,0.1220,0.120,126.985,267.50
1,Juice WRLD,Face 2 Face,0.720,0.711,-4.532,0.09480,0.0308,0.1010,0.437,155.997,124.56
2,ROSALÍA,DESPECHÁ RMX,0.582,0.954,-5.744,0.23500,0.0727,0.0572,0.884,130.008,170.83
3,Only The Family,Mad Max,0.640,0.898,-8.796,0.02120,0.2550,0.0919,0.680,149.988,153.60
4,Little Simz,Gorilla,0.813,0.716,-5.748,0.26900,0.2560,0.1040,0.682,90.418,245.82
...,...,...,...,...,...,...,...,...,...,...,...
95,young friend,little pieces,0.576,0.564,-9.626,0.21700,0.1310,0.6740,0.677,79.896,157.50
96,chemical club,Bounds,0.713,0.531,-7.337,0.00118,0.0324,0.3280,0.692,150.043,207.07
97,whiterosemoxie,whts rght,0.509,0.444,-9.560,0.34500,0.0881,0.1040,0.237,113.780,258.00
98,Mokita,High,0.570,0.669,-6.364,0.14400,0.0558,0.2660,0.299,75.029,166.83
